In [12]:
#Import all important libary for this analysis
#This file contains only the best methods tested in preprocessing analysis
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Read the SPSS Merged Median house price dataset which is available on my drive
#This data already contains no missing value and part of the preprocessing has been done in spss
df = pd.read_csv("Dataset/MergedMedianHousePrice_Deprivation_Compact1_NonMissing.csv",header=0,delimiter=',')


In [14]:
#Keep only required columns for this analysis
#all other years are taken away because we only want to make use of the most recent year.
columns_to_keep = ['LSOAcode2011', 'LSOAname', 'YearendingDec2021', 'IndexofMultipleDeprivationIMDScore', 'IncomeScorerate', 'EmploymentScorerate', 'EducationSkillsandTrainingScore', 'HealthDeprivationandDisabilityScore', 'CrimeScore', 'BarrierstoHousingandServicesScore', 'LivingEnvironmentScore', 'IncomeDeprivationAffectingChildrenIndexIDACIScorerate', 'IncomeDeprivationAffectingOlderPeopleIDAOPIScorerate', 'ChildrenandYoungPeopleSubdomainScore', 'AdultSkillsSubdomainScore', 'GeographicalBarriersSubdomainScore', 'WiderBarriersSubdomainScore', 'IndoorsSubdomainScore', 'OutdoorsSubdomainScore']
df = df[columns_to_keep]
column_mapping = {
    'LSOAcode2011': 'LSOACode',
    'LSOAname': 'LSOAName',
    'YearendingDec2021': 'HousePrice', 
    'IndexofMultipleDeprivationIMDScore': 'IMDScore',
    'IncomeScorerate': 'IncDepriv', 
    'EmploymentScorerate': 'EmpDepriv',  
    'EducationSkillsandTrainingScore': 'EduSklDepriv',
    'HealthDeprivationandDisabilityScore': 'HealthDepriv',
    'CrimeScore': 'CrimScore',
    'BarrierstoHousingandServicesScore': 'HousServDepriv', 
    'LivingEnvironmentScore': 'LivEnvDepriv', 
    'IncomeDeprivationAffectingChildrenIndexIDACIScorerate': 'ChildIncDepriv',
    'IncomeDeprivationAffectingOlderPeopleIDAOPIScorerate': 'OldPplIncDepriv',
    'ChildrenandYoungPeopleSubdomainScore': 'ChildYPDepriv', 
    'AdultSkillsSubdomainScore': 'AdultSklDepriv',
    'GeographicalBarriersSubdomainScore': 'GeoBarDepriv',
    'WiderBarriersSubdomainScore': 'WiderBarDepriv',  
    'IndoorsSubdomainScore': 'IndoorDepriv',  
    'OutdoorsSubdomainScore': 'OutdoorDepriv'  
}

# Now rename the columns in the DataFrame
df.rename(columns=column_mapping, inplace=True)



In [15]:
#Lets start by simply grouping our column names so we can access them easily at any point in time.
allFeatures = [] # : all features would be stored here
allNumericFeatures = [] # : all features to be normalized or all numeric features
featuresNormalized = [] # : all features wwill eventually be normalized into these designated columns these columns 
                            #would have _n to their names
for column in df.columns:
    if df[column].dtype == 'float64':  # Check that type is float
        #Tis feature has to be normalized
            featuresNormalized.append(column+"_n")
            allNumericFeatures.append(column)

    allFeatures.append(column) #Stores all the columns or features of these dataset : wether it is old or freshly created


#I am verifying I have what into expected in my respective variables
print("\nfeatures that would be normalized ")
print(allNumericFeatures)
print("\nfeatures would be normalized into these columns : ")
print(featuresNormalized)
print("\nThese all all the features of this data set: ")
print(allFeatures)




features that would be normalized 
['HousePrice', 'IMDScore', 'IncDepriv', 'EmpDepriv', 'EduSklDepriv', 'HealthDepriv', 'CrimScore', 'HousServDepriv', 'LivEnvDepriv', 'ChildIncDepriv', 'OldPplIncDepriv', 'ChildYPDepriv', 'AdultSklDepriv', 'GeoBarDepriv', 'WiderBarDepriv', 'IndoorDepriv', 'OutdoorDepriv']

features would be normalized into these columns : 
['HousePrice_n', 'IMDScore_n', 'IncDepriv_n', 'EmpDepriv_n', 'EduSklDepriv_n', 'HealthDepriv_n', 'CrimScore_n', 'HousServDepriv_n', 'LivEnvDepriv_n', 'ChildIncDepriv_n', 'OldPplIncDepriv_n', 'ChildYPDepriv_n', 'AdultSklDepriv_n', 'GeoBarDepriv_n', 'WiderBarDepriv_n', 'IndoorDepriv_n', 'OutdoorDepriv_n']

These all all the features of this data set: 
['LSOACode', 'LSOAName', 'HousePrice', 'IMDScore', 'IncDepriv', 'EmpDepriv', 'EduSklDepriv', 'HealthDepriv', 'CrimScore', 'HousServDepriv', 'LivEnvDepriv', 'ChildIncDepriv', 'OldPplIncDepriv', 'ChildYPDepriv', 'AdultSklDepriv', 'GeoBarDepriv', 'WiderBarDepriv', 'IndoorDepriv', 'OutdoorDep

# Removing outliers using Interquartile Range IQR 

In [20]:
#Build function to mark rows with outliers
columns_with_outliers = allNumericFeatures
def outliers(dfc, ft, scale):
    Q1 = dfc[ft].quantile(0.25)
    Q3 = dfc[ft].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - scale * IQR
    upper_bound = Q3 + scale * IQR
    
    ls = df.index[ (dfc[ft] < lower_bound) | (dfc[ft] > upper_bound)]
    print("feature name : "+ft)
    print("Number of outliers : "+str(len(sorted(set(ls)))))
    print("third quartile "+str(Q3))
    print("first quartile "+str(Q3))
    print("Inter Quartile Range "+str(IQR))
    print("\--------------------------- ")
    return ls
    

    
    
    
#Build function to remove rows marked as outliers
    
def removeOutlierIndex(df,ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

In [21]:
#Call function to mark rows with outliers
index_list = []
for feature in columns_with_outliers:
    index_list.extend(outliers(df,feature,0.5))

feature name : HousePrice
Number of outliers : 4349
third quartile 400000.0
first quartile 400000.0
Inter Quartile Range 205005.0
\--------------------------- 
feature name : IMDScore
Number of outliers : 3646
third quartile 25.133000000000003
first quartile 25.133000000000003
Inter Quartile Range 16.283500000000004
\--------------------------- 
feature name : IncDepriv
Number of outliers : 3679
third quartile 0.169
first quartile 0.169
Inter Quartile Range 0.10800000000000001
\--------------------------- 
feature name : EmpDepriv
Number of outliers : 4070
third quartile 0.136
first quartile 0.136
Inter Quartile Range 0.07700000000000001
\--------------------------- 
feature name : EduSklDepriv
Number of outliers : 3791
third quartile 26.490499999999997
first quartile 26.490499999999997
Inter Quartile Range 19.975499999999997
\--------------------------- 
feature name : HealthDepriv
Number of outliers : 5033
third quartile 0.418
first quartile 0.418
Inter Quartile Range 1.107
\--------

In [22]:
#Cal function to remove outlier rows
df_no_outliers = removeOutlierIndex(df,index_list)

In [23]:
df_no_outliers.shape

(7510, 19)

# Now Normalzing the data

In [24]:
#Normalzie all the designated features to normalize here
#When we are normalizing we want to keep the original data untouched so we normalize into a different column
dfp = df_no_outliers #where dfp is dataframe processed
dfp[featuresNormalized] = dfp[allNumericFeatures].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

print("finished preprocessing.....")


finished preprocessing.....
